### Ver2
* VIF 분석을 통해 피처 선택을 수행하고 테스트합니다.
* VIF 분석 결과 제거할 피처가 없어 permutation_importance 함수를 사용 후 제거할 피쳐를 선택합니다.
* Perform feature selection through VIF analysis and test it.
* After conducting the VIF analysis and finding no features to remove, I will now use the permutation_importance function to determine which features to remove.

In [1]:
# !pip install git+https://github.com/tooha289/DataAnalysisLibrary.git

  Cloning https://github.com/tooha289/DataAnalysisLibrary.git to c:\users\medici\appdata\local\temp\pip-req-build-w_7176q7
  Resolved https://github.com/tooha289/DataAnalysisLibrary.git to commit e65d47d5b1b85d6d4f87376519b594cd106a596d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/tooha289/DataAnalysisLibrary.git 'C:\Users\medici\AppData\Local\Temp\pip-req-build-w_7176q7'


# 사용 모듈 Import

In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from DataAnalysis import eda
from DataAnalysis import feature_engineering

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.callbacks import EarlyStopping

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score

# 1. Load & Check Data

In [71]:
df = pd.read_csv('../data/train.csv')
df

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136424,136424,M22284,M,300.1,311.4,1530,37.5,210,0,0,0,0,0,0
136425,136425,H38017,H,297.5,308.5,1447,49.1,2,0,0,0,0,0,0
136426,136426,L54690,L,300.5,311.8,1524,38.5,214,0,0,0,0,0,0
136427,136427,L53876,L,301.7,310.9,1447,46.3,42,0,0,0,0,0,0


In [72]:
test_df = pd.read_csv('../data/test.csv')
test_df

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,136429,L50896,L,302.3,311.5,1499,38.0,60,0,0,0,0,0
1,136430,L53866,L,301.7,311.0,1713,28.8,17,0,0,0,0,0
2,136431,L50498,L,301.3,310.4,1525,37.7,96,0,0,0,0,0
3,136432,M21232,M,300.1,309.6,1479,47.6,5,0,0,0,0,0
4,136433,M19751,M,303.4,312.3,1515,41.3,114,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90949,227378,L51130,L,302.3,311.4,1484,40.4,15,0,0,0,0,0
90950,227379,L47783,L,297.9,309.8,1542,33.8,31,0,0,0,0,0
90951,227380,L48097,L,295.6,306.2,1501,41.4,187,0,0,0,0,0
90952,227381,L48969,L,298.1,307.8,1534,40.3,69,0,0,0,0,0


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136429 entries, 0 to 136428
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       136429 non-null  int64  
 1   Product ID               136429 non-null  object 
 2   Type                     136429 non-null  object 
 3   Air temperature [K]      136429 non-null  float64
 4   Process temperature [K]  136429 non-null  float64
 5   Rotational speed [rpm]   136429 non-null  int64  
 6   Torque [Nm]              136429 non-null  float64
 7   Tool wear [min]          136429 non-null  int64  
 8   Machine failure          136429 non-null  int64  
 9   TWF                      136429 non-null  int64  
 10  HDF                      136429 non-null  int64  
 11  PWF                      136429 non-null  int64  
 12  OSF                      136429 non-null  int64  
 13  RNF                      136429 non-null  int64  
dtypes: f

In [74]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,136429.0,NaN,NaN,NaN,68214.0,39383.804275,0.0,34107.0,68214.0,102321.0,136428.0
Product ID,136429,9976,L53257,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Type,136429,3,L,95354,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Air temperature [K],136429.0,NaN,NaN,NaN,299.862776,1.862247,295.3,298.3,300.0,301.2,304.4
Process temperature [K],136429.0,NaN,NaN,NaN,309.94107,1.385173,305.8,308.7,310.0,310.9,313.8
Rotational speed [rpm],136429.0,NaN,NaN,NaN,1520.33111,138.736632,1181.0,1432.0,1493.0,1580.0,2886.0
Torque [Nm],136429.0,NaN,NaN,NaN,40.348643,8.502229,3.8,34.6,40.4,46.1,76.6
Tool wear [min],136429.0,NaN,NaN,NaN,104.408901,63.96504,0.0,48.0,106.0,159.0,253.0
Machine failure,136429.0,NaN,NaN,NaN,0.015744,0.124486,0.0,0.0,0.0,0.0,1.0
TWF,136429.0,NaN,NaN,NaN,0.001554,0.039389,0.0,0.0,0.0,0.0,1.0


In [75]:
df.isnull().sum()

id                         0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

# 2. Separate columns

In [76]:
feature_cols = df.columns.difference(['id', 'Product ID', 'Machine failure'])
numeric_cols = df.columns.difference(['id', 'Product ID', 'Machine failure', 'Type'])
continuos_cols = feature_cols.difference(["TWF", "HDF", "PWF", "OSF", "RNF", "Type"])
discrete_cols = feature_cols.difference(continuos_cols)

# 3. FeatureEngineering

## 3.1 Preprocessing

In [77]:
dfp = feature_engineering.DataFramePreprocessor()
le = LabelEncoder()
stds = StandardScaler()

### 3.1.1 Train set

In [78]:
x_data = df.copy()
x_data = x_data.drop(labels=['id', 'Product ID'], axis=1)
x_data, _ = dfp.fit_transform_multiple_transformer(x_data, [le, stds], [["Type"], continuos_cols])
x_data = pd.concat([x_data, df[discrete_cols.difference(['Type'])]], axis=1)

x_data[discrete_cols] = x_data[discrete_cols].astype("int8")
x_data

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Tool wear [min],Torque [Nm],HDF,OSF,PWF,RNF,TWF
0,1,0.395880,-0.246230,0.545416,0.556417,-0.499711,0,0,0,0,0
1,2,1.469856,1.558605,1.720308,1.494433,-1.323028,0,0,0,0,0
2,1,-0.302204,-1.040358,2.051873,-1.241447,-1.628831,0,0,0,0,0
3,1,0.610675,0.692284,0.026445,1.447532,0.464745,0,0,0,0,0
4,2,-1.000288,-0.679391,0.869773,-1.100744,-0.582043,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
136424,2,0.127386,1.053251,0.069693,1.650769,-0.335048,0,0,0,0,0
136425,0,-1.268781,-1.040358,-0.528565,-1.601020,1.029305,0,0,0,0,0
136426,1,0.342182,1.342025,0.026445,1.713303,-0.217431,0,0,0,0,0
136427,1,0.986567,0.692284,-0.528565,-0.975676,0.699979,0,0,0,0,0


In [79]:
x_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136429 entries, 0 to 136428
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Type                     136429 non-null  int8   
 1   Air temperature [K]      136429 non-null  float64
 2   Process temperature [K]  136429 non-null  float64
 3   Rotational speed [rpm]   136429 non-null  float64
 4   Tool wear [min]          136429 non-null  float64
 5   Torque [Nm]              136429 non-null  float64
 6   HDF                      136429 non-null  int8   
 7   OSF                      136429 non-null  int8   
 8   PWF                      136429 non-null  int8   
 9   RNF                      136429 non-null  int8   
 10  TWF                      136429 non-null  int8   
dtypes: float64(5), int8(6)
memory usage: 6.0 MB


In [80]:
y_data = df.copy().pop('Machine failure')
y_data = y_data.astype("int8")
y_data

0         0
1         0
2         0
3         0
4         0
         ..
136424    0
136425    0
136426    0
136427    0
136428    0
Name: Machine failure, Length: 136429, dtype: int8

### 3.1.2 Test set

In [81]:
x_test = test_df.copy()
x_test = x_test.drop(labels=['id', 'Product ID'], axis=1)
x_test, _ = dfp.fit_transform_multiple_transformer(x_test, [le, stds], [["Type"], continuos_cols])
x_test = pd.concat([x_test, test_df[discrete_cols.difference(['Type'])]], axis=1)

x_test[discrete_cols] = x_test[discrete_cols].astype("int8")
x_test

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Tool wear [min],Torque [Nm],HDF,OSF,PWF,RNF,TWF
0,1,1.313830,1.126570,-0.153806,-0.693494,-0.274579,0,0,0,0,0
1,1,0.990824,0.765635,1.375097,-1.366729,-1.356341,0,0,0,0,0
2,1,0.775487,0.332512,0.031949,-0.129855,-0.309854,0,0,0,0,0
3,2,0.129475,-0.244985,-0.296694,-1.554608,0.854217,0,0,0,0,0
4,2,1.906008,1.704068,-0.039496,0.151964,0.113445,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
90949,1,1.313830,1.054383,-0.260972,-1.398042,0.007620,0,0,0,0,0
90950,1,-1.054879,-0.100611,0.153403,-1.147536,-0.768427,0,0,0,0,0
90951,1,-2.293068,-2.699348,-0.139517,1.294897,0.125203,0,0,0,0,0
90952,1,-0.947211,-1.544354,0.096248,-0.552584,-0.004138,0,0,0,0,0


In [82]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90954 entries, 0 to 90953
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Type                     90954 non-null  int8   
 1   Air temperature [K]      90954 non-null  float64
 2   Process temperature [K]  90954 non-null  float64
 3   Rotational speed [rpm]   90954 non-null  float64
 4   Tool wear [min]          90954 non-null  float64
 5   Torque [Nm]              90954 non-null  float64
 6   HDF                      90954 non-null  int8   
 7   OSF                      90954 non-null  int8   
 8   PWF                      90954 non-null  int8   
 9   RNF                      90954 non-null  int8   
 10  TWF                      90954 non-null  int8   
dtypes: float64(5), int8(6)
memory usage: 4.0 MB


## 3.2 Feature selecting

### 3.2.1 VIF analysis

In [15]:
fs = feature_engineering.FeatureSelector()

In [16]:
formula_cols = pd.Series(feature_cols).apply(lambda col: f"Q('{col}')")
formula_cols

0         Q('Air temperature [K]')
1                         Q('HDF')
2                         Q('OSF')
3                         Q('PWF')
4     Q('Process temperature [K]')
5                         Q('RNF')
6      Q('Rotational speed [rpm]')
7                         Q('TWF')
8             Q('Tool wear [min]')
9                 Q('Torque [Nm]')
10                       Q('Type')
dtype: object

In [17]:
formula = "Q('Machine failure')~" + "+".join(formula_cols)
formula+="-1"
formula

"Q('Machine failure')~Q('Air temperature [K]')+Q('HDF')+Q('OSF')+Q('PWF')+Q('Process temperature [K]')+Q('RNF')+Q('Rotational speed [rpm]')+Q('TWF')+Q('Tool wear [min]')+Q('Torque [Nm]')+Q('Type')-1"

In [18]:
data = pd.concat([x_data, y_data], axis=1)

In [19]:
fs.get_vif_dataframe(formula, data)

,features,VIF Factor
0,Q('Air temperature [K]'),3.806145
4,Q('Process temperature [K]'),3.772470
9,Q('Torque [Nm]'),2.629339
6,Q('Rotational speed [rpm]'),2.608535
1,Q('HDF'),1.038264
3,Q('PWF'),1.037175
2,Q('OSF'),1.030903
10,Q('Type'),1.011599
8,Q('Tool wear [min]'),1.006448
7,Q('TWF'),1.006107


### 3.2.2 Get permutation importance

In [20]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.25, stratify=y_data)

In [86]:
dtc = DecisionTreeClassifier()
dtc.fit(x_train.values, y_train.values)

DecisionTreeClassifier()

In [87]:
fs.get_permutation_importance(dtc, x_valid, y_valid)

,feature,perm_importance
0,Type,-0.000183
1,Air temperature [K],0.007430
2,Process temperature [K],0.008772
3,Rotational speed [rpm],0.002276
4,Tool wear [min],0.000863
5,Torque [Nm],0.002767
6,HDF,0.008867
7,OSF,0.006846
8,PWF,0.003044
9,RNF,0.000033


* 위 결과를 바탕으로 우선 RNF값을 제거하고 성능을 평가합니다.
* 그 후 Type에 대해서도 제거를 수행해봅니다.

# 4. Modeling

## 4.1 LogisticRegression

In [20]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.25, stratify=y_data)

In [21]:
lr = LogisticRegression()
lr.fit(x_train.values, y_train.values)

LogisticRegression()

In [22]:
acc_score = lr.score(x_train.values, y_train)
acc_score

0.9960320950733476

In [23]:
f1score = f1_score(y_train, lr.predict(x_train.values))
f1score

0.8568406205923836

In [24]:
acc_score = lr.score(x_valid.values, y_valid)
acc_score

0.996393807904304

In [25]:
f1score = f1_score(y_valid, lr.predict(x_valid.values))
f1score

0.8717413972888425

## 4.2 XGBoost

In [83]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = []
train_score = []
valid_score = []
for train_index, test_index in skf.split(x_data, y_data):
    x_train, x_valid = x_data.iloc[train_index], x_data.iloc[test_index]
    y_train, y_valid = y_data.iloc[train_index], y_data.iloc[test_index]
    
    xgb = XGBClassifier()
    xgb.fit(x_train.values, y_train.values, verbose=2)
    
    acc_score = xgb.score(x_train, y_train)
    predict_val = xgb.predict(x_train)
    f1score = f1_score(y_train, predict_val)
    roc_auc = roc_auc_score(y_train, predict_val)
    train_score.append((acc_score, f1score, roc_auc))
    
    acc_score = xgb.score(x_valid, y_valid)
    predict_val = xgb.predict(x_valid)
    f1score = f1_score(y_valid, predict_val)
    roc_auc = roc_auc_score(y_valid, predict_val)
    valid_score.append((acc_score, f1score, roc_auc))
    model.append(xgb)

In [84]:
for acc, f1, roc in valid_score:
    print(f"Accuracy: {acc}, f1_score: {f1}, Roc-Auc: {roc}")
np.array(valid_score).mean(axis=0)

Accuracy: 0.9960052774316499, f1_score: 0.8593548387096774, Roc-Auc: 0.8878698655479385
Accuracy: 0.9960052774316499, f1_score: 0.858257477243173, Roc-Auc: 0.8835533699108423
Accuracy: 0.9965916587260867, f1_score: 0.8824273072060682, Roc-Auc: 0.9055905397260844
Accuracy: 0.9957120867844316, f1_score: 0.845442536327609, Roc-Auc: 0.8719626985611462
Accuracy: 0.9958218801539307, f1_score: 0.8523316062176166, Roc-Auc: 0.8831892340605477


array([0.99602724, 0.85956275, 0.88643314])

* 모든 데이터 사용시

In [ ]:
# Accuracy: 0.9960052774316499, f1_score: 0.8593548387096774, Roc-Auc: 0.8878698655479385
# Accuracy: 0.9960052774316499, f1_score: 0.858257477243173, Roc-Auc: 0.8835533699108423
# Accuracy: 0.9965916587260867, f1_score: 0.8824273072060682, Roc-Auc: 0.9055905397260844
# Accuracy: 0.9957120867844316, f1_score: 0.845442536327609, Roc-Auc: 0.8719626985611462
# Accuracy: 0.9958218801539307, f1_score: 0.8523316062176166, Roc-Auc: 0.8831892340605477
# array([0.99602724, 0.85956275, 0.88643314])

In [27]:
x_data.drop('RNF', axis=1, inplace=True)
x_test.drop('RNF', axis=1, inplace=True)

* RNF 제거 시 스코어

In [28]:
# Accuracy: 0.9960052774316499, f1_score: 0.8597168597168596, Roc-Auc: 0.8890167495931356
# Accuracy: 0.9960419262625523, f1_score: 0.859375, Roc-Auc: 0.8835719877243662
# Accuracy: 0.9966649563878912, f1_score: 0.883780332056194, Roc-Auc: 0.9021952567006809
# Accuracy: 0.9956021402917247, f1_score: 0.841688654353562, Roc-Auc: 0.8707626722364238
# Accuracy: 0.9958218801539307, f1_score: 0.8523316062176166, Roc-Auc: 0.8831892340605477
# array([0.99602724, 0.85937849, 0.88574718])

In [31]:
# x_data.drop('Type', axis=1, inplace=True)
# x_test.drop('Type', axis=1, inplace=True)

* 추가로 Type 제거 시 스코어

In [32]:
# Accuracy: 0.9960419262625523, f1_score: 0.8608247422680411, Roc-Auc: 0.8890353667134395
# Accuracy: 0.9960785750934545, f1_score: 0.8601307189542484, Roc-Auc: 0.8824464326537399
# Accuracy: 0.9966649563878912, f1_score: 0.8834827144686299, Roc-Auc: 0.9010510838165307
# Accuracy: 0.9955288426299201, f1_score: 0.8390501319261213, Roc-Auc: 0.8695812637252253
# Accuracy: 0.995968480850284, f1_score: 0.8567708333333334, Roc-Auc: 0.8832637053146436
# array([0.99605656, 0.86005183, 0.88507557])

### 4.2.1 Create submission.csv

In [36]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.25, stratify=y_data)

In [37]:
xgb = XGBClassifier()
xgb.fit(x_train.values, y_train.values, verbose=2)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [38]:
prediction = xgb.predict_proba(x_test)[:,1]
len(prediction)

90954

In [39]:
result = test_df.copy()
result = result.drop(result.columns.difference(['id']), axis=1)
result['Machine failure'] = prediction
result

,id,Machine failure
0,136429,0.000132
1,136430,0.017592
2,136431,0.000046
3,136432,0.000893
4,136433,0.003733
...,...,...
90949,227378,0.000348
90950,227379,0.000465
90951,227380,0.000708
90952,227381,0.000178


In [40]:
result.to_csv('../data/submission.csv', index = False)

## 4.3 TensorFlow

In [56]:
# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.001, restore_best_weights=True)

In [61]:
# 모델 구조 정의하기 --- (※3)
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(x_data.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [62]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 128)               1280      
                                                                 
 dense_21 (Dense)            (None, 128)               16512     
                                                                 
 dense_22 (Dense)            (None, 64)                8256      
                                                                 
 dense_23 (Dense)            (None, 1)                 65        
                                                                 
 activation_5 (Activation)   (None, 1)                 0         
                                                                 
Total params: 26,113
Trainable params: 26,113
Non-trainable params: 0
_________________________________________________________________


In [63]:
model.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid), callbacks=[early_stopping])

Epoch 1/100
3198/3198 [==============================] - 6s 2ms/step - loss: 0.0331 - binary_accuracy: 0.9941 - val_loss: 0.0225 - val_binary_accuracy: 0.9963
Epoch 2/100
3198/3198 [==============================] - 5s 2ms/step - loss: 0.0231 - binary_accuracy: 0.9961 - val_loss: 0.0238 - val_binary_accuracy: 0.9961
Epoch 3/100
3198/3198 [==============================] - 5s 2ms/step - loss: 0.0226 - binary_accuracy: 0.9961 - val_loss: 0.0219 - val_binary_accuracy: 0.9963
Epoch 4/100
3198/3198 [==============================] - 5s 2ms/step - loss: 0.0222 - binary_accuracy: 0.9961 - val_loss: 0.0226 - val_binary_accuracy: 0.9962
Epoch 5/100
3198/3198 [==============================] - 5s 2ms/step - loss: 0.0220 - binary_accuracy: 0.9961 - val_loss: 0.0219 - val_binary_accuracy: 0.9962
Epoch 6/100
3198/3198 [==============================] - 5s 2ms/step - loss: 0.0217 - binary_accuracy: 0.9961 - val_loss: 0.0221 - val_binary_accuracy: 0.9962
Epoch 7/100
3198/3198 [=======================

In [64]:
predicted_probabilities = model.predict(x_valid)
predicted_classes = (predicted_probabilities > 0.5).astype(int)

1066/1066 [==============================] - 1s 816us/step


In [65]:
f1score = f1_score(y_valid, predicted_classes)
f1score

0.8682008368200836

In [66]:
roc_auc = roc_auc_score(y_valid, predicted_classes)
print("ROC-AUC Score:", roc_auc)

ROC-AUC Score: 0.8863463838030374


### 4.3.1 Create submission.csv

In [67]:
predicted_probabilities = model.predict(x_test)
len(prediction)

2843/2843 [==============================] - 2s 840us/step


90954

In [68]:
result = test_df.copy()
result = result.drop(result.columns.difference(['id']), axis=1)
result['Machine failure'] = predicted_probabilities
result

,id,Machine failure
0,136429,0.000350
1,136430,0.000671
2,136431,0.000438
3,136432,0.001524
4,136433,0.000441
...,...,...
90949,227378,0.000625
90950,227379,0.000770
90951,227380,0.001427
90952,227381,0.000510


In [69]:
result.to_csv('../data/submission.csv', index = False)